<a href="https://colab.research.google.com/github/imy0ung/AI_Basic_Study/blob/main/Pytorch/Pytorch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **역전파**

* 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 Backpropagtion 입니다.
* Backpropagtation은 가중치를 업데이트하는데 사용합니다.
* SGD,SGD-Momentum,AdaGrad,RmsProp, Adam... 을 통해 최적의 가중치를 찾는데, 기울기를 찾으려면 chain-rule이 필요하고, activation function이 있다면 한 perceptron당 3번의 미분이 필요합니다. => gradient를 업데이트
* 딥러닝 시스템은 매우 많은 perceptron이 존재하기에,각각 3번의 미분을 하는건 매우 많은 연산이 필요합니다.
* 그래서 Pytorch에서는 torch.augograd라고 불리는 자동 미분 엔진이 내장되어 있음. 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.

# **Autograd**
* Pytorch의 'autograd'는 계산 그래프를 동적으로 생성하고, 역전파를 토오해 각 연산의 기울기를 자동으로 계산합니다.
* 'autograd'는 각 연산에 대한 그래디언트를 저장하고, 이를 기반으로 chain-rule을 사용하여 전체 그래프의 기울기를 효율적으로 계산합니다.

### **Gradient 계산하기**

In [5]:
import torch

x = torch.ones(5) # input
y = torch.zeros(3) # output
w = torch.randn(5,3,requires_grad= True) # weight, requires_grad : loss function의 변화도를 알기 위한 변수
b = torch.randn(3, requires_grad = True) # bias
z = torch.matmul(x,w) + b # z = wx + nb
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y) # loss-function

In [6]:
# grad_fn : 해당 텐서가 어떤 연산에 의해 생성되었는지 나타내는 속성. 계산 그래프를 추적
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7d0f83776440>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7d0f83774d90>


In [7]:
# Gradient 계산하기
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3265, 0.2140, 0.0496],
        [0.3265, 0.2140, 0.0496],
        [0.3265, 0.2140, 0.0496],
        [0.3265, 0.2140, 0.0496],
        [0.3265, 0.2140, 0.0496]])
tensor([0.3265, 0.2140, 0.0496])


### **변화도 추적 멈추기**
기본적으로 requires_grad = True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우, 추적이나 지원이 필요 없을 수 있습니다. 이러한 경우, 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있습니다.

In [10]:
z = torch.matmul(x,w) + b
print(z.requires_grad) # gradient 구할 수 있는지 확인

with torch.no_grad() :
    z = torch.matmul(x,w) + b

print(z.requires_grad) # gradient 구할 수 있는지 확인

True
False


In [11]:
z = torch.matmul(x,w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


# **AutoGrad에 대한 추가정보**

* 개념적으로 autograd는 텐서가 실행된 모든 연산들의 기록을 Function 객체로 구성된 DAG에 저장합니다.

* 순전파 단계에서, autograd는 다음 두 가지 작업을 동시에 수행합니다
    1. 요청된 연산을 수행하여 결과 텐서를 반환하고,
    2. DAG에 연산의 gradient function을 유지합니다.

* 역전파 단계에서, .backward()가 호출될 때 시작됩니다.
    1. 각 .grad_fn으로부터 변화도를 추적/계산하고
    2. 각 텐서의 .grad 속성에 계산 결과를 쌓고
    3. chain-rule을 사용하여, 모든 텐서들까지 propagate 합니다.